<a href="https://colab.research.google.com/github/SamLiaoP/eval_llm_trans_pydocs_zhtw/blob/main/eval_LLM_translation_python_docs_zh_tw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: 安裝套件

In [10]:
from google.colab import userdata

# 如果要比較GPT的話，請在左邊的 Secret 欄位中放入 OPEN_AI_API_KEY
# 名稱 OPEN_AI_API_KEY 值，則是Token
OPEN_AI_API_KEY = userdata.get('OPEN_AI_API_KEY')

In [3]:
!apt-get install pciutils lshw
!pip install openai ollama chromadb pandas

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids usb.ids
The following NEW packages will be installed:
  libpci3 lshw pci.ids pciutils usb.ids
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 883 kB of archives.
After this operation, 3,256 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 883 kB in 2s (486 kB/s)
Selecting previously unselected package pci.ids.
(Reading d

In [4]:
# 記得要開GPU!!! 不然ollama會跑死
# 可以按右上方的 變更執行階段類型 選擇  T4GPU
!nvidia-smi

Sun Nov  3 10:25:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Step 2: 試跑ollama

In [5]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13320    0 13320    0     0  36524      0 --:--:-- --:--:-- --:--:-- 36593
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [29]:
# ollama serve in the background
# 如果後續的服務有 暫停 的話，一定要重跑這行，不然有可能一直報錯，但查不出來。
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama pull cwchang/llama-3-taiwan-8b-instruct

In [8]:
!ollama run cwchang/llama-3-taiwan-8b-instruct "請介紹自己"

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ 您好！很高興認識您。我是一個由台灣團隊開發的人工智慧助理，專門協助華語用戶解決各種問題。無論是生活中的大小疑難雜症，或是工作、學習上需要幫忙的地方，我都非常樂意提供協助。

我的知識涵蓋廣泛的領域，包括但不限於歷史、文化、科學、科技等。同時我也擁有獨特的人文關懷，致力於以同理心理解每個用戶的需求，並給出最真誠、實用的建議。

此外，我還具備強大的語言能力，不僅能流利地使用繁體中文與您溝通，也能根據上下文靈活轉換簡體中文或其他方言，讓我們的交流更加自然順暢。無論是口頭對話還是書面撰寫，我都可以勝任不同型別的文字互動。

作為您的AI夥伴，我希望能成為您生活和工作中的得力助手，不僅幫您解決問題，更要陪伴在您身邊，成為值得信賴的朋友。如果有任何需要幫忙的地方，歡迎隨時告訴我，我將竭誠為您服務。



In [9]:
import ollama
response = ollama.chat(model='cwchang/llama-3-taiwan-8b-instruct', messages=[
  {
    'role': 'user',
    'content': '為什麼天空是藍色的？',
  },
])
print(response['message']['content'])

當太陽的白光穿過地球的大氣層時，會被氮氧分子等粒子散射。藍色光的波長比其他顏色短，更容易被散射，因此成為最多的顏色並在天空中可見。


# Step 3: 定義基礎LLM接口

如果要連線其他Model也可以自己做一個類似的Interface，只要有query的功能，且input 為純文字即可。

In [11]:
class LLMModelInterface:
    """
    LLMModelInterface 為基礎接口類別，用於擴展不同模型的查詢方法。
    這個類別定義了 query 方法，讓所有繼承的類別都可以實現該方法來與 LLM 模型溝通。
    """
    def query(self, system_prompt: str, user_prompt: str, model: str, temperature: float = 0.1):
        """
        抽象方法，必須在子類別中實現，用於查詢 LLM 模型並返回回應。

        Args:
            system_prompt (str): 系統提示，提供背景或上下文。
            user_prompt (str): 使用者的查詢內容。
            model (str): 模型名稱。
            temperature (float): 控制回應的隨機性，範圍 0 到 1。

        Returns:
            str: 模型的回應文字。
        """
        raise NotImplementedError("LLM Model interface needs implementation.")


In [12]:
import openai
from openai import OpenAI, APIError

class OpenAIModel(LLMModelInterface):
    """
    OpenAIModel 利用 OpenAI GPT 模型來生成回答。

    Attributes:
        api_key (str): OpenAI 的 API 金鑰。
    """

    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)

    def query(self, system_prompt: str, user_prompt: str, model: str, temperature: float = 0.1):
        """
        查詢 OpenAI 模型並返回生成的回應。

        Args:
            system_prompt (str): 系統提示，提供背景或上下文。
            user_prompt (str): 使用者的查詢內容。
            model (str): 使用的 OpenAI 模型名稱。
            temperature (float): 控制回應的隨機性，範圍 0 到 1。

        Returns:
            str: OpenAI 模型生成的回應。
        """
        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temperature
            )
            return response.choices[0].message.content.strip()
        except APIError as e:
            return f"Error occurred: {e}"


In [13]:
import ollama

class OllamaModel(LLMModelInterface):
    """
    OllamaModel 利用 Ollama 模型來生成回答。
    """

    def query(self, system_prompt: str, user_prompt: str, model: str, temperature: float = 0.1):
        """
        查詢 Ollama 模型並返回生成的回應。

        Args:
            system_prompt (str): 系統提示，提供背景或上下文。
            user_prompt (str): 使用者的查詢內容。
            model (str): 使用的 Ollama 模型名稱。
            temperature (float): 控制回應的隨機性，範圍 0 到 1。

        Returns:
            str: Ollama 模型生成的回應。
        """
        # only for jupyter notebook
        # print(system_prompt)
        response = ollama.chat(model=model, messages=[
        {
            'role': 'system',
            'content': system_prompt,
        },
        {
            'role': 'user',
            'content': user_prompt,
        },
        ])
        return response['message']['content']


#  Step 4: 設置不同Prompt(目前只有一個)

## System Prompt by Rainy

因為覺得 Rainy 的Prompt很完整，就直接拿來稍微修改一下就用了ＸＤ

In [14]:
system_prompt_by_Rainy = """
[@1] 工作描述
你的工作是將官方的python英文手冊翻譯為繁體中文(臺灣)，你將先獲得完整的原文以了解其語境，並接著獲得指定的一段RST格式英文字串，請在兼顧語境、用詞用字(稍後在[@2]及[@3]段落描述)及合理的RST語法(稍後在[@4]段落描述)的前提下將指定的一段RST格式英文字串翻譯為RST格式繁體中文(臺灣)字串。

[@2] 用詞用字
基本概念:
中文句使用全形標點符號；英文句維持半形的標點符號。
部分較為罕見的專有名詞或普遍使用原文描述的名詞可能會使用原文(譯文)的格式進行翻譯或甚至不翻譯，詳情請參照[@3]術語表。
務必保留 reStructuredText (RST) 格式的正確使用，RST語法注意事項稍後在[@4]段落描述。
中英文交雜時，中英文間要插入空白；全形中文標點符號與英文間則不用。
專有名詞應該參考下方術語表[@3]進行翻譯。

[@3] 術語表
以下術語表使用
<英文> -> <繁體中文(臺灣)>
格式進行術語表列
```
" " -> 「 」
( ) -> （ ）
, -> ，
. -> 。
abstract base class -> 抽象基底類別
annotation -> 註釋
approximate -> 近似
argument -> 引數
asynchronous -> 非同步
attribute -> 屬性
awaitable -> 可等待物件
binary file -> 二進位檔案
binary framed protocol -> 二進位分框協定
boolean -> 布林（boolean）
borrowed reference -> 借用參照
bytes-like object -> 類位元組串物件（bytes-like object）
bytecode -> 位元組碼（bytecode）
callable -> 可呼叫物件（callable）
callback -> 回呼
child -> 子- / 下代
cipher -> 加密方法
class -> 類別
complex number -> 複數
condition -> 條件
context -> 情境
contributor -> 貢獻者
column -> 欄 / 行 （column）
  要特別注意簡體與繁體中文的用法相反，正確的繁體中文（台灣）翻譯為直行（column）
coroutine -> 協程（coroutine）
custom -> 自訂
CPython -> CPython
decorator -> 裝飾器（decorator）
deprecated -> 已棄用
descriptor -> 描述器（descriptor）
deserialization -> 去序列化（deserialization）
dict -> 字典（dict）
dictionary -> 字典（dictionary）
dictionary comprehension -> 字典綜合運算（dictionary comprehension）
dispatch table -> 調度表
docstring -> 說明字串
docstring -> 鴨子型別（duck-typing）
element -> 元素
escape -> 轉義
  僅指 ascii 的 escape character 時使用「轉義」，否則譯做「跳脫」
evaluate -> 給值 / 計算
  需根據前後文決定
exception -> 例外
expression -> 運算式
extension module -> 擴充模組（extension module）
f-string -> f 字串
file-like object -> 類檔案物件
finalizing / finalize -> 最終化
finder -> 尋檢器（finder）
flag -> 旗標
float -> 浮點數（float）
floor division -> 向下取整除法
function -> 函式
garbage collection -> 垃圾回收（garbage collection）
generator -> 產生器
generic function -> 泛型函式（generic function）
generic type -> 泛型型別（generic type）
GIL -> 全域直譯器鎖 （GIL）
global -> 全域
hash -> 雜湊
helper -> 幫助函式、輔助函式
identity -> 識別性
import -> 引入（import）
immutable -> 不可變物件（immutable）
index -> 索引
instance -> 實例
int -> 整數（int）
interpreter -> 直譯器
introspection -> 自省
iterable -> 可疊代物件（iterable）
iterate -> 疊代
iterator -> 疊代器
key function -> 鍵函數（key function）
keyword argument -> 關鍵字引數（keyword argument）
kwarg -> 關鍵字引數（kwarg）
lambda -> lambda
level -> 階 / 層級 / 層
  現有中文翻譯資料都有將層級與階混用的情形，需根據前後文決定翻譯內容，例如 High-level （高階），Top-level （頂層）等等
library -> 函式庫
list -> 串列（list）
list comprehension -> 串列綜合運算（list comprehension）
local -> 區域
loop -> 迴圈
magic method -> 魔術方法（magic method）
metaclass -> 元類別（metaclass）
method -> 方法（method）
mock -> mock
module -> 模組（module）
object -> 物件
opcode -> 操作碼
operand -> 運算元
operator -> 運算子
package -> 套件
parameter -> 參數
parent -> 父- / 上代
parse -> 剖析
parser -> 剖析器
patch -> patch
PEP -> PEP
pickle -> pickle 封包
 作為名詞使用時保留封包的敘述
pickle -> 封裝
 作為動詞使用時譯為封裝
pickler -> 封裝器（pickler）
pickling -> 封裝
policy -> 政策 / 原則
  若使用語境為“使用別的公司的policy”，可以翻譯為「政策」，其餘狀況則應翻譯為「原則」
prompt -> 提示字元
qualified name -> 限定名稱
reduce -> 縮減
reduction function -> 縮減函式
return -> 回傳
reference count -> 參照計數（reference count）
row -> 列（row）
  要特別注意簡體與繁體中文的用法相反，正確的繁體中文（台灣）翻譯為橫列（row）
sequence -> 序列（sequence）
set -> 集合（set）
signature -> 簽名
signature -> 輸入特徵
  特指python的函數輸入參數特徵時才使用輸入特徵作為翻譯
slice -> 切片
statement -> 陳述式
support -> 支援
type -> 型別
unpickle -> 拆封
unpickler -> 拆封器（unpickler）
wrapper -> 包裝器
wrapper function -> 包裝函數
```

[@4] RST 特殊語法注意事項
## 關於雙斜線的使用時機

在翻譯字串中的 rst 特殊語法 (e.g.: mod:\`os\` ) 旁有時會需要空格才能正常建置，但當不想在網頁 (html) 上顯示空格時就會需要用到雙斜線 `\\ `。

以譯文 `參閱 os 模組` 為例：

|PO 譯文寫法 | 備註|
|-|-|
|參閱 :mod:\`os\` 模組 | 這是正常寫法，os 部分會變成超連結|
|參閱:mod:\`os\` 模組 | build failed: sphinx 認不出特殊語法|
|參閱:mod:\`os\`模組 | build failed: sphinx 認不出特殊語法|
|參閱 :mod:\`os\`模組 | build failed: sphinx 認不出特殊語法|
|參閱\\ :mod:\`os\` 模組 | build 成功，因為前有 `\\ ` 空出特殊語法的間隔，讓sphinx可以正確辨識語法，但`\\ `會將該空格字元視覺顯示上的空格消去，但顯示上中英文間應要有空格|

有時候特殊語法是可能 render 出中文字的

|PO 譯文寫法|備註|
|-|-|
|一個 :term:\`file object\`。 | `file object` 部分變成超連結|
|一個\\ :term:\`檔案物件 \<file object\>\`。 | `檔案物件`部分變成超連結，且不希望中文字間有空白|
|參考 \`wiki 文章 <https://wiki.com/...>`_\\ 中 | `wiki 文章`最後是中文字，文字間的銜接不希望顯示空白，故特殊語法加上 `\\ ` 以消除視覺上空格|

## 關於 rST 的常見問題
翻譯文件的時候，在遇到 rST 語法的時候，經常遇到一些問題，這個章節就各種例外狀態做一個統整

遇到**全型逗號**、**全型句號**、**全型冒號**等標點符號時，可以正常使用 rST 語法，我們可以觀察上個章節的例子：

一個 :term:\`file object\`。

本例子的 rST 特殊語法 :term:\`file object\` ，在遇到全型句號時可以照正常的規則使用，即可以根據狀況單純使用空格或不使用空格隔開 rST 語法與標點符號。

但如果標點符號是**全型括號**時，就會引發錯誤：

|PO 譯文寫法|備註|
|-|-|
|一個 :term:\`file object\`（ | build failed |
|一個 :term:\`file object\`\\（ | build passed |

因此在翻譯途中，若是遇到全型括號與 rST 語法同時出現時，就需要特別注意。

## 更簡潔的 rST 的 literal block 標記語法

po file 中看到原文以 `::` 結尾時，只要像以下這樣翻譯，就能顯示全形冒號並且同時成功標記接下來的段落是一個 literal block：

```
msgid "blah blah::"
msgstr "blah blah： ::"
```
也就是當原文以 `::` 結尾時，譯文內使用 `： ::`（全形冒號x1 + 空格 x1 + 半形冒號x2）就可以了。


user content是你應當翻譯的RST原文(英文)輸入，請在兼顧語境"
([@5]段落的完整原文可供參考)、用詞用字(已經在[@2]及
[@3]段落描述過)及合理的RST語法(已經在[@4]段落描述過
)的前提下將其翻譯為符合RST格式的繁體中文(臺灣)字串
"""

## System Prompt List
函數會全部都跑過一次，如果有新的Prompt可以定義在此dict中

In [15]:
# 定義prompt字典，可以一次測試多個prompt
prompt_dict = {
    "system-prompt-by-Rainy": system_prompt_by_Rainy
    # "YOUR_OWN_PROMPT": super awesome prompt.
}


# Step 5:定義生成翻譯結果的函數

In [16]:
def generate_translations_df(df, models, system_prompt_dict):
    """
    為 DataFrame 中的每個 msgid 欄位產生不同模型和翻譯風格的翻譯結果。

    Args:
        df (DataFrame): 包含 msgid 欄位的 DataFrame。
        models (dict): 模型字典，包含模型名稱和實例。
        system_prompt_dict(dict): 包含要遍歷的prompt

    Returns:
        DataFrame: 包含翻譯結果的 DataFrame。
    """
    results_df = df.copy()
    for model_name, model in models.items():
        for style, system_prompt in system_prompt_dict.items():
            col_name = f"{style}_{model_name}"
            print(col_name)
            results_df[col_name] = results_df.apply(
                lambda row: model.query(system_prompt, row["msgid"], model_name), axis=1
            )
    return results_df


# Step 6:載入從專案中擷取出來的翻譯資料集

Commit SHA = "0313a637a30633bbb32082494cf30daad16ff7a7"
committed on Sep 24

抓出所有 msgid 和 msgstr 以及對應的 file path

In [18]:
import pandas as pd
# 此份檔案儲存於Git Hub 中
df = pd.read_csv("po_extracted_data.csv")
df = df.dropna(axis=0, how='any')

In [19]:
df_sample = df.sample(n=100, random_state=123)
df_sample.reset_index(drop = True, inplace=True)

In [20]:
df_sample

,msgid,msgstr,file_path
0,2,2,library/spwd.po
1,The modules described in this chapter provide ...,本章節所描述的模組 (module) 提供了多樣的專門資料型別，例如日期與時間、固定型別陣列...,library/datatypes.po
2,":pep:`647`, User-Defined Type Guards",:pep:`647`，使用者定義的型別防護 (User-Defined Type Guards),whatsnew/3.10.po
3,:class:`tzinfo` Objects,:class:`tzinfo` 物件,library/datetime.po
4,true,true,library/stdtypes.po
...,...,...,...
95,"iso2022jp-ext, iso-2022-jp-ext","iso2022jp-ext, iso-2022-jp-ext",library/codecs.po
96,Arabic,阿拉伯文,library/codecs.po
97,A class in an :keyword:`except` clause matches...,一個在 :keyword:`except` 子句中的 class（類別）和一個例外是可相容的...,tutorial/errors.po
98,f-string,f-string（f 字串）,reference/lexical_analysis.po


# Step 6:生成翻譯結果

In [ ]:
# 這個是經過繁體中文Fine-Tune的Llama3模型
!ollama pull cwchang/llama-3-taiwan-8b-instruct
!ollama pull gemma2:9b

In [30]:
# 初始化模型
openai_model = OpenAIModel(api_key=OPEN_AI_API_KEY)
ollama_model = OllamaModel()

# 定義模型字典
models = {
    "cwchang/llama-3-taiwan-8b-instruct": ollama_model,
    "gemma2:9b": ollama_model,
    "gpt-4o-mini": openai_model
}

# 生成翻譯結果 DataFrame
df_translate = generate_translations_df(df_sample, models, prompt_dict)
df_translate


system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct
system-prompt-by-Rainy_gemma2:9b
system-prompt-by-Rainy_gpt-4o-mini


,msgid,msgstr,file_path,system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct,system-prompt-by-Rainy_gemma2:9b,system-prompt-by-Rainy_gpt-4o-mini
0,2,2,library/spwd.po,## 關於可迭代物件\n\n可迭代物件是 Python 中的一個重要概念，它指的是可以用 f...,You said the number 2. \n\nWhat can I do with ...,抱歉，我需要您提供要翻譯的RST原文內容，才能進行翻譯。請提供相關的英文內容。
1,The modules described in this chapter provide ...,本章節所描述的模組 (module) 提供了多樣的專門資料型別，例如日期與時間、固定型別陣列...,library/datatypes.po,本章所描述的模組提供各種專門用途的資料型別，例如日期和時間、固定型陣列、堆疊佇列、二端點佇列...,This statement describes the core functionalit...,本章所描述的模組提供各種專用的資料型別，例如日期和時間、固定型別的陣列、堆積佇列、雙端佇列以...
2,":pep:`647`, User-Defined Type Guards",:pep:`647`，使用者定義的型別防護 (User-Defined Type Guards),whatsnew/3.10.po,[:pep:`647`，使用者定義型別守衛](https://www.python.org/...,"Let's dive into PEP 647, which introduces a po...",:pep:`647`，使用者定義型別保護器
3,:class:`tzinfo` Objects,:class:`tzinfo` 物件,library/datetime.po,：class:``tzinfo`` Objects,Let's explore `tzinfo` objects in Python's dat...,:class:`tzinfo` 物件
4,true,true,library/stdtypes.po,"你好,我很樂意幫助你將 Python 官方手冊中的英文 RST 原文翻譯成繁體中文。請提供要...",Is there anything else I can help you with? 😊 ...,抱歉，我無法協助滿足該要求。
...,...,...,...,...,...,...
95,"iso2022jp-ext, iso-2022-jp-ext","iso2022jp-ext, iso-2022-jp-ext",library/codecs.po,"ISO-2022-JP-EXT, ISO-2022-JP-EXT","You're referring to **ISO 2022 JP-EXT**, a cha...",iso2022jp-ext，iso-2022-jp-ext
96,Arabic,阿拉伯文,library/codecs.po,阿拉伯語（Arabic）,Please tell me what you want to do with Arabic...,抱歉，我無法協助滿足該要求。
97,A class in an :keyword:`except` clause matches...,一個在 :keyword:`except` 子句中的 class（類別）和一個例外是可相容的...,tutorial/errors.po,在：keyword:`except` 子句中列出的類別會匹配該類別或其衍生類別的實例（但反之...,You're absolutely right! \n\nLet's illustrate ...,在 :keyword:`except` 陳述式中，類別會匹配該類別本身或其衍生類別的例外（但...
98,f-string,f-string（f 字串）,reference/lexical_analysis.po,f-字串,Let's explore f-strings in Python!\n\n**What a...,f 字串


In [31]:
df_translate.to_csv("trans_without_rag.csv", index=False)

# Step 7:評估機制

1. 詞序相似度 (Sequence Similarity)
詞序相似度衡量兩個文本的詞語順序是否相似。使用 SequenceMatcher，將翻譯和參考譯文分詞後進行比較，計算兩者的相似比例（值介於 0 到 100 之間）。分數越高表示翻譯結果和參考譯文的詞序相似度較高。

2. 詞彙重疊 (Vocabulary Overlap)
詞彙重疊指標量化翻譯和參考譯文之間的共同詞彙數量。通過計算兩者之間的交集來確定重疊率，並根據兩者獨特詞數的較大值進行標準化（結果為百分比）。較高的詞彙重疊率表明翻譯和參考譯文在用詞上有較高的一致性。

3. 長度比 (Length Ratio)
長度比是衡量翻譯結果和參考譯文之間長度的相似度。它通過兩者的字符數進行比較，計算兩者長度比的較小值（結果為百分比）。這一指標確保翻譯結果和參考譯文的長度差異不大，較高的分數表示長度相近。

4. BLEU 分數 (BLEU Score)
BLEU（Bilingual Evaluation Understudy）是一種常用的機器翻譯質量指標，量化翻譯結果和參考譯文之間的詞組匹配程度。BLEU 分數越高，說明翻譯結果在詞組匹配上越接近參考譯文。

5. CHRF 分數 (CHRF Score)
CHRF 分數結合了字符和詞的 n-gram 匹配，特別適用於評估翻譯結果和參考譯文的語境和流暢度。它通過字符 n-gram 匹配來量化翻譯結果的連貫性和一致性。較高的 CHRF 分數表明翻譯結果與參考譯文在表達上更一致。

6. 總分 (Overall Score)
總分是各指標的加權平均值，用於綜合評估翻譯質量。每個指標按特定權重（20%）進行計算，最終的總分反映翻譯結果和參考譯文的整體相似度。


By ChatGPT 整理

In [32]:
import jieba
from difflib import SequenceMatcher
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.chrf_score import sentence_chrf
import pandas as pd


In [33]:
def evaluate_translation(translation, reference):
    """
    使用多種指標評估翻譯結果與參考譯文的相似度。

    Args:
        translation (str): 翻譯結果。
        reference (str): 參考譯文。

    Returns:
        dict: 各指標的得分以及綜合總分。
    """
    trans_words = jieba.lcut(translation)
    ref_words = jieba.lcut(reference)

    # 計算各指標
    scores = {
        "詞序相似度": SequenceMatcher(None, trans_words, ref_words).ratio() * 100,
        "詞彙重疊": len(set(trans_words) & set(ref_words)) / max(len(set(trans_words)), len(set(ref_words))) * 100,
        "長度比": min(len(translation) / len(reference), len(reference) / len(translation)) * 100,
        "BLEU": sentence_bleu([ref_words], trans_words) * 100,
        "CHRF": sentence_chrf(ref_words, trans_words) * 100,
    }

    # 綜合評分
    scores["總分"] = sum([scores["詞序相似度"] * 0.2, scores["詞彙重疊"] * 0.2, scores["長度比"] * 0.2,
                        scores["BLEU"] * 0.2, scores["CHRF"] * 0.2])
    return scores


In [34]:
def evaluate_translations_in_df(df):
    """
    對 DataFrame 中的翻譯結果欄位進行評估，並將各指標得分添加為新欄位。

    Args:
        df (DataFrame): 包含翻譯結果的 DataFrame。

    Returns:
        DataFrame: 含原始欄位與評估得分的 DataFrame。
    """
    results_df = df.copy()
    reference_col = "msgstr"
    translation_cols = [col for col in df.columns if col.startswith(("system-prompt-by-Rainy"))]

    for col_name in translation_cols:
        score_data = results_df.apply(lambda row: evaluate_translation(row[col_name], row[reference_col]), axis=1)
        score_df = pd.DataFrame(score_data.tolist()).add_prefix(f"{col_name}___")
        results_df = pd.concat([results_df, score_df], axis=1)

    return results_df


In [35]:
def calculate_average_scores(df):
    """
    計算每個模型和提示字串的翻譯結果的平均評估分數。

    Args:
        df (DataFrame): 包含翻譯結果和評估分數的 DataFrame。

    Returns:
        list: 每個模型和提示字串的平均評估分數。
    """
    model_prompt_pairs = {col.split("___")[0] for col in df.columns if col.endswith("___總分")}
    average_scores = []
    print(model_prompt_pairs)

    for model_prompt in model_prompt_pairs:
        score_columns = [col for col in df.columns if col.startswith(f"{model_prompt}___")]

        # 僅篩選出數值型欄位
        numeric_score_columns = df[score_columns].select_dtypes(include='number')

        # 計算每個評估指標的平均分數
        avg_scores = numeric_score_columns.mean().to_dict()

        # 移除欄位前綴並添加模型與提示字串名稱
        cleaned_avg_scores = {col.replace(f"{model_prompt}___", ""): score for col, score in avg_scores.items()}
        cleaned_avg_scores['模型和提示字串'] = model_prompt
        average_scores.append(cleaned_avg_scores)

    return average_scores

In [36]:
# 對每個翻譯結果進行評估並計算平均分數
evaluated_df = evaluate_translations_in_df(df_translate)
average_scores = calculate_average_scores(evaluated_df)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.775 seconds.
DEBUG:jieba:Loading model cost 0.775 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lowe

{'system-prompt-by-Rainy_gpt-4o-mini', 'system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct', 'system-prompt-by-Rainy_gemma2:9b'}


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [37]:
average_scores

[{'詞序相似度': 69.28054802447923,
  '詞彙重疊': 66.55910073068158,
  '長度比': 79.46049492346789,
  'BLEU': 47.025029141796196,
  'CHRF': 59.69040437232802,
  '總分': 64.40311543855059,
  '模型和提示字串': 'system-prompt-by-Rainy_gpt-4o-mini'},
 {'詞序相似度': 50.828013916976424,
  '詞彙重疊': 52.084118132399965,
  '長度比': 67.72756100909713,
  'BLEU': 22.618326551942914,
  'CHRF': 43.69895608314751,
  '總分': 47.39139513871278,
  '模型和提示字串': 'system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct'},
 {'詞序相似度': 4.630253626473911,
  '詞彙重疊': 5.016399164801918,
  '長度比': 4.401061998890552,
  'BLEU': 1.5038966899631705,
  'CHRF': 12.578451106258928,
  '總分': 5.6260125172776965,
  '模型和提示字串': 'system-prompt-by-Rainy_gemma2:9b'}]

# Step 8:加入 RAG

In [38]:
# Embedding 用 Model
!ollama pull all-minilm

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 797b70c4edf8...   0% ▕▏    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% ▕▏    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% ▕▏    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% ▕▏    0 B/ 45 MB                  pulling man

## 建立 Vector Database

In [39]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

class VectorProcessor:
    """
    VectorProcessor 負責將文本轉換為向量並儲存到 ChromaDB，並支援檢索相關的文本。

    Attributes:
        model_name (str): 用於向量化的模型名稱。
        chroma_path (str): ChromaDB 的儲存路徑。
        collection_name (str): ChromaDB 中的集合名稱。
    """

    def __init__(self, model_name="all-minilm", chroma_path=".", collection_name="default"):
        self.model_name = model_name
        # 初始化 Chroma 客戶端
        self.chroma_client = chromadb.PersistentClient(path=chroma_path)
        # 設定 Ollama 嵌入函數
        self.ollama_ef = embedding_functions.OllamaEmbeddingFunction(
            url="http://localhost:11434/api/embeddings",
            model_name=self.model_name
        )
        # 創建或獲取 Chroma Collection
        self.collection = self.chroma_client.get_or_create_collection(
            name=collection_name,
            embedding_function=self.ollama_ef
        )

    def process_msgid(self, df):
        """
        將 DataFrame 中的 msgid 向量化並儲存至 ChromaDB 中。

        Args:
            df (DataFrame): 包含 msgid 和 file_path 欄位的 DataFrame。
        """
        count = 0
        for _, row in df.iterrows():
            print(f"{count}/{len(df)}")
            count += 1
            self.collection.add(
                documents=[row["msgid"]],
                metadatas=[{"file_path": row["file_path"], "msgstr": row["msgstr"]}],
                ids=[str(int(row.name)+1)]  # 使用行索引作為 ID
            )
        return self.collection

    def retrieve_relevant_msgid(self, user_prompt, n_results=5):
        """
        從 ChromaDB 中檢索與 user_prompt 最相關的 msgid，並返回對應的 msgstr 和 msgid。

        Args:
            user_prompt (str): 要查詢的提示文字。
            n_results (int): 返回的相關結果數量。

        Returns:
            list: 包含查詢結果的 msgid 和 msgstr。
        """
        query_results = self.collection.query(
            query_texts=[user_prompt],
            n_results=n_results,
            include=["documents", "metadatas", "distances"]
        )

        return query_results

    def delete_collection(self, collection_name="default"):
        self.chroma_client.delete_collection(name=collection_name)



In [40]:
# 拿掉df中的df_sample的rows
df_remaining = df.merge(df_sample, how="outer", indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)


# 由於GPU限制，只拿df_remaining中的10000筆做Vector Database
df_remaining = df_remaining[0:10000]

# 初始化 VectorProcessor 並將 DataFrame 中的 msgid 向量化存儲
vector_processor = VectorProcessor()
vector_processor.process_msgid(df_remaining)

# vector_processor.delete_collection()

串流輸出內容已截斷至最後 5000 行。
5000/10000
5001/10000
5002/10000
5003/10000
5004/10000
5005/10000
5006/10000
5007/10000
5008/10000
5009/10000
5010/10000
5011/10000
5012/10000
5013/10000
5014/10000
5015/10000
5016/10000
5017/10000
5018/10000
5019/10000
5020/10000
5021/10000
5022/10000
5023/10000
5024/10000
5025/10000
5026/10000
5027/10000
5028/10000
5029/10000
5030/10000
5031/10000
5032/10000
5033/10000
5034/10000
5035/10000
5036/10000
5037/10000
5038/10000
5039/10000
5040/10000
5041/10000
5042/10000
5043/10000
5044/10000
5045/10000
5046/10000
5047/10000
5048/10000
5049/10000
5050/10000
5051/10000
5052/10000
5053/10000
5054/10000
5055/10000
5056/10000
5057/10000
5058/10000
5059/10000
5060/10000
5061/10000
5062/10000
5063/10000
5064/10000
5065/10000
5066/10000
5067/10000
5068/10000
5069/10000
5070/10000
5071/10000
5072/10000
5073/10000
5074/10000
5075/10000
5076/10000
5077/10000
5078/10000
5079/10000
5080/10000
5081/10000
5082/10000
5083/10000
5084/10000
5085/10000
5086/10000
5087/10000
5088/10000


Collection(id=e8e875a4-98a8-4851-9986-88163ce3d60c, name=default)

In [41]:
df_remaining

,msgid,msgstr,file_path
0,0\n\n ...,0\n\n ...,library/heapq.po
1,Apache License\n...,Apache License\n...,license.po
2,list of:\n+------+...,list of:\n+------+...,library/doctest.po
3,6\n ...,6\n ...,howto/mro.po
4,x ** (alpha - 1) * math.exp(-x / bet...,x ** (alpha - 1) * math.exp(-x / bet...,library/random.po
...,...,...,...
10035,Example patterns for using the :class:`ChainMa...,用 :class:`ChainMap` 類別模擬巢狀上下文的範例模式： ::,library/collections.po
10036,Example raising an exception after SHORT_TIMEO...,在 SHORT_TIMEOUT 秒後引發例外的範例： ::,library/test.po
10037,Example usage of :class:`ModuleFinder`,:class:`ModuleFinder` 的用法範例,library/modulefinder.po
10038,Example usage of this module could be as follo...,該模組的範例用法如下： ::,library/__main__.po


In [42]:
# 查詢最相關的 msgid，並獲取相應的 msgstr 和 msgid 最近的5筆
user_prompt = "Python is a mature programming language which has established a reputation for stability.  In order to maintain this reputation, the developers would like to know of any deficiencies you find in Python."
relevant_results = vector_processor.retrieve_relevant_msgid(user_prompt, n_results=5)

In [43]:
# 提取結果中的 msgid 和 msgstr
results = []
for doc, metadata in zip(relevant_results['documents'][0], relevant_results['metadatas'][0]):
    results.append({"msgid": doc, "msgstr": metadata["msgstr"]})
results

[{'msgid': 'During the 1989 Christmas holidays, I had a lot of time on my hand, so I decided to give it a try.  During the next year, while still mostly working on it in my own time, Python was used in the Amoeba project with increasing success, and the feedback from colleagues made me add many early improvements.',
  'msgstr': '在 1989 年的聖誕節假期，我有很多自由時間，所以我決定來嘗試一下。在接下來的一年裡，雖然我大部分時間仍然在為此而努力，但 Python 在 Amoeba 專案中的使用得到了越來越多的成功，且同事們的回饋也使我為它增加了許多早期的改進。'},
 {'msgid': '$ python3.13\nPython 3.13 (default, April 4 2023, 09:25:04)\n[GCC 10.2.0] on linux\nType "help", "copyright", "credits" or "license" for more information.\n>>>',
  'msgstr': '$ python3.12\nPython 3.12 (default, April 4 2022, 09:25:04)\n[GCC 10.2.0] on linux\nType "help", "copyright", "credits" or "license" for more information.\n>>>'},
 {'msgid': 'Cameron Laird maintains a useful set of pages about Python web technologies at https://web.archive.org/web/20210224183619/http://phaseit.net/claird/comp.lang.python/web_python.',
  'ms

In [44]:
# 如果你好奇...這裡是每一個 msgid 對應的文件 path
results = []
for doc, metadata in zip(relevant_results['documents'][0], relevant_results['metadatas'][0]):
    results.append({"msgid": doc, "msgstr": metadata["msgstr"], "file_path": metadata["file_path"]})
results

[{'msgid': 'During the 1989 Christmas holidays, I had a lot of time on my hand, so I decided to give it a try.  During the next year, while still mostly working on it in my own time, Python was used in the Amoeba project with increasing success, and the feedback from colleagues made me add many early improvements.',
  'msgstr': '在 1989 年的聖誕節假期，我有很多自由時間，所以我決定來嘗試一下。在接下來的一年裡，雖然我大部分時間仍然在為此而努力，但 Python 在 Amoeba 專案中的使用得到了越來越多的成功，且同事們的回饋也使我為它增加了許多早期的改進。',
  'file_path': 'faq/general.po'},
 {'msgid': '$ python3.13\nPython 3.13 (default, April 4 2023, 09:25:04)\n[GCC 10.2.0] on linux\nType "help", "copyright", "credits" or "license" for more information.\n>>>',
  'msgstr': '$ python3.12\nPython 3.12 (default, April 4 2022, 09:25:04)\n[GCC 10.2.0] on linux\nType "help", "copyright", "credits" or "license" for more information.\n>>>',
  'file_path': 'tutorial/interpreter.po'},
 {'msgid': 'Cameron Laird maintains a useful set of pages about Python web technologies at https://web.archive.org/web/20

## RAG需要的prompt

我把 Rainy 的 Prompt 拆成兩部分，並且把 [@6] 加入，代表 Vector Databse 回傳的內容

In [45]:
system_prompt_by_Rainy_part_1_for_rag = """
[@1] 工作描述
你的工作是將官方的python英文手冊翻譯為繁體中文(臺灣)，你將先獲得完整的原文以了解其語境，並接著獲得指定的一段RST格式英文字串，請在兼顧語境、用詞用字(稍後在[@2]及[@3]段落描述)及合理的RST語法(稍後在[@4]段落描述)的前提下將指定的一段RST格式英文字串翻譯為RST格式繁體中文(臺灣)字串。

[@2] 用詞用字
基本概念:
中文句使用全形標點符號；英文句維持半形的標點符號。
部分較為罕見的專有名詞或普遍使用原文描述的名詞可能會使用原文(譯文)的格式進行翻譯或甚至不翻譯，詳情請參照[@3]術語表。
務必保留 reStructuredText (RST) 格式的正確使用，RST語法注意事項稍後在[@4]段落描述。
中英文交雜時，中英文間要插入空白；全形中文標點符號與英文間則不用。
專有名詞應該參考下方術語表[@3]進行翻譯。

[@3] 術語表
以下術語表使用
<英文> -> <繁體中文(臺灣)>
格式進行術語表列
```
" " -> 「 」
( ) -> （ ）
, -> ，
. -> 。
abstract base class -> 抽象基底類別
annotation -> 註釋
approximate -> 近似
argument -> 引數
asynchronous -> 非同步
attribute -> 屬性
awaitable -> 可等待物件
binary file -> 二進位檔案
binary framed protocol -> 二進位分框協定
boolean -> 布林（boolean）
borrowed reference -> 借用參照
bytes-like object -> 類位元組串物件（bytes-like object）
bytecode -> 位元組碼（bytecode）
callable -> 可呼叫物件（callable）
callback -> 回呼
child -> 子- / 下代
cipher -> 加密方法
class -> 類別
complex number -> 複數
condition -> 條件
context -> 情境
contributor -> 貢獻者
column -> 欄 / 行 （column）
  要特別注意簡體與繁體中文的用法相反，正確的繁體中文（台灣）翻譯為直行（column）
coroutine -> 協程（coroutine）
custom -> 自訂
CPython -> CPython
decorator -> 裝飾器（decorator）
deprecated -> 已棄用
descriptor -> 描述器（descriptor）
deserialization -> 去序列化（deserialization）
dict -> 字典（dict）
dictionary -> 字典（dictionary）
dictionary comprehension -> 字典綜合運算（dictionary comprehension）
dispatch table -> 調度表
docstring -> 說明字串
docstring -> 鴨子型別（duck-typing）
element -> 元素
escape -> 轉義
  僅指 ascii 的 escape character 時使用「轉義」，否則譯做「跳脫」
evaluate -> 給值 / 計算
  需根據前後文決定
exception -> 例外
expression -> 運算式
extension module -> 擴充模組（extension module）
f-string -> f 字串
file-like object -> 類檔案物件
finalizing / finalize -> 最終化
finder -> 尋檢器（finder）
flag -> 旗標
float -> 浮點數（float）
floor division -> 向下取整除法
function -> 函式
garbage collection -> 垃圾回收（garbage collection）
generator -> 產生器
generic function -> 泛型函式（generic function）
generic type -> 泛型型別（generic type）
GIL -> 全域直譯器鎖 （GIL）
global -> 全域
hash -> 雜湊
helper -> 幫助函式、輔助函式
identity -> 識別性
import -> 引入（import）
immutable -> 不可變物件（immutable）
index -> 索引
instance -> 實例
int -> 整數（int）
interpreter -> 直譯器
introspection -> 自省
iterable -> 可疊代物件（iterable）
iterate -> 疊代
iterator -> 疊代器
key function -> 鍵函數（key function）
keyword argument -> 關鍵字引數（keyword argument）
kwarg -> 關鍵字引數（kwarg）
lambda -> lambda
level -> 階 / 層級 / 層
  現有中文翻譯資料都有將層級與階混用的情形，需根據前後文決定翻譯內容，例如 High-level （高階），Top-level （頂層）等等
library -> 函式庫
list -> 串列（list）
list comprehension -> 串列綜合運算（list comprehension）
local -> 區域
loop -> 迴圈
magic method -> 魔術方法（magic method）
metaclass -> 元類別（metaclass）
method -> 方法（method）
mock -> mock
module -> 模組（module）
object -> 物件
opcode -> 操作碼
operand -> 運算元
operator -> 運算子
package -> 套件
parameter -> 參數
parent -> 父- / 上代
parse -> 剖析
parser -> 剖析器
patch -> patch
PEP -> PEP
pickle -> pickle 封包
 作為名詞使用時保留封包的敘述
pickle -> 封裝
 作為動詞使用時譯為封裝
pickler -> 封裝器（pickler）
pickling -> 封裝
policy -> 政策 / 原則
  若使用語境為“使用別的公司的policy”，可以翻譯為「政策」，其餘狀況則應翻譯為「原則」
prompt -> 提示字元
qualified name -> 限定名稱
reduce -> 縮減
reduction function -> 縮減函式
return -> 回傳
reference count -> 參照計數（reference count）
row -> 列（row）
  要特別注意簡體與繁體中文的用法相反，正確的繁體中文（台灣）翻譯為橫列（row）
sequence -> 序列（sequence）
set -> 集合（set）
signature -> 簽名
signature -> 輸入特徵
  特指python的函數輸入參數特徵時才使用輸入特徵作為翻譯
slice -> 切片
statement -> 陳述式
support -> 支援
type -> 型別
unpickle -> 拆封
unpickler -> 拆封器（unpickler）
wrapper -> 包裝器
wrapper function -> 包裝函數
```

[@4] RST 特殊語法注意事項
## 關於雙斜線的使用時機

在翻譯字串中的 rst 特殊語法 (e.g.: mod:\`os\` ) 旁有時會需要空格才能正常建置，但當不想在網頁 (html) 上顯示空格時就會需要用到雙斜線 `\\ `。

以譯文 `參閱 os 模組` 為例：

|PO 譯文寫法 | 備註|
|-|-|
|參閱 :mod:\`os\` 模組 | 這是正常寫法，os 部分會變成超連結|
|參閱:mod:\`os\` 模組 | build failed: sphinx 認不出特殊語法|
|參閱:mod:\`os\`模組 | build failed: sphinx 認不出特殊語法|
|參閱 :mod:\`os\`模組 | build failed: sphinx 認不出特殊語法|
|參閱\\ :mod:\`os\` 模組 | build 成功，因為前有 `\\ ` 空出特殊語法的間隔，讓sphinx可以正確辨識語法，但`\\ `會將該空格字元視覺顯示上的空格消去，但顯示上中英文間應要有空格|

有時候特殊語法是可能 render 出中文字的

|PO 譯文寫法|備註|
|-|-|
|一個 :term:\`file object\`。 | `file object` 部分變成超連結|
|一個\\ :term:\`檔案物件 \<file object\>\`。 | `檔案物件`部分變成超連結，且不希望中文字間有空白|
|參考 \`wiki 文章 <https://wiki.com/...>`_\\ 中 | `wiki 文章`最後是中文字，文字間的銜接不希望顯示空白，故特殊語法加上 `\\ ` 以消除視覺上空格|

## 關於 rST 的常見問題
翻譯文件的時候，在遇到 rST 語法的時候，經常遇到一些問題，這個章節就各種例外狀態做一個統整

遇到**全型逗號**、**全型句號**、**全型冒號**等標點符號時，可以正常使用 rST 語法，我們可以觀察上個章節的例子：

一個 :term:\`file object\`。

本例子的 rST 特殊語法 :term:\`file object\` ，在遇到全型句號時可以照正常的規則使用，即可以根據狀況單純使用空格或不使用空格隔開 rST 語法與標點符號。

但如果標點符號是**全型括號**時，就會引發錯誤：

|PO 譯文寫法|備註|
|-|-|
|一個 :term:\`file object\`（ | build failed |
|一個 :term:\`file object\`\\（ | build passed |

因此在翻譯途中，若是遇到全型括號與 rST 語法同時出現時，就需要特別注意。

## 更簡潔的 rST 的 literal block 標記語法

po file 中看到原文以 `::` 結尾時，只要像以下這樣翻譯，就能顯示全形冒號並且同時成功標記接下來的段落是一個 literal block：

```
msgid "blah blah::"
msgstr "blah blah： ::"
```
也就是當原文以 `::` 結尾時，譯文內使用 `： ::`（全形冒號x1 + 空格 x1 + 半形冒號x2）就可以了。
"""

system_prompt_by_Rainy_part_2_for_rag = """
user content是你應當翻譯的RST原文(英文)輸入，請在兼顧語境"
[@6]其他相關的文章，以及對應翻譯可供參考。
([@5]段落的完整原文可供參考)、用詞用字(已經在[@2]及
[@3]段落描述過)及合理的RST語法(已經在[@4]段落描述過
)的前提下將其翻譯為符合RST格式的繁體中文(臺灣)字串
"""

## 定義生成RAG翻譯結果的函數

In [46]:
def generate_rag_translations_df(df, models, system_prompt_dict, vector_processor):
    """
    為 DataFrame 中的每個 msgid 欄位產生不同模型和翻譯風格的檢索增強生成（RAG）翻譯結果。

    Args:
        df (DataFrame): 包含 msgid 欄位的 DataFrame。
        models (dict): 模型字典，包含模型名稱和實例。
        system_prompt_dict (dict): 各種提示字串的字典。
        vector_processor (VectorProcessor): 向量處理器，用於檢索相關的內容。

    Returns:
        DataFrame: 包含翻譯結果的 DataFrame。
    """
    results_rag_df = df.copy()  # 複製原始 DataFrame

    for model_name, model in models.items():
        for style, base_system_prompt in system_prompt_dict.items():
            col_name = f"{style}_{model_name}_RAG"
            print(f"Generating column: {col_name}")

            def generate_rag_prompt(row):
                # 使用 row["msgid"] 進行檢索
                relevant_results = vector_processor.retrieve_relevant_msgid(row["msgid"], n_results=3)

                # 提取相關內容的 msgid 和 msgstr
                results = []
                for doc, metadata in zip(relevant_results['documents'][0], relevant_results['metadatas'][0]):
                    results.append({"msgid": doc, "msgstr": metadata["msgstr"]})

                # 組合檢索到的內容為完整的提示字串
                relevant_texts = "\n".join([f"[@6]相關文章內容: {res['msgid']}\n對應中文翻譯: {res['msgstr']}" for res in results])
                full_system_prompt = f"{system_prompt_by_Rainy_part_1_for_rag}\n\n{relevant_texts}\n\n{system_prompt_by_Rainy_part_2_for_rag}"
                # print(full_system_prompt)

                return full_system_prompt

            # 將包含檢索內容的 system_prompt 傳入模型進行翻譯
            results_rag_df[col_name] = results_rag_df.apply(
                lambda row: model.query(generate_rag_prompt(row), row["msgid"], model_name), axis=1
            )

    return results_rag_df


In [47]:
# 初始化模型
openai_model = OpenAIModel(api_key=OPEN_AI_API_KEY)
ollama_model = OllamaModel()

# 定義模型字典
models = {
    "cwchang/llama-3-taiwan-8b-instruct": ollama_model,
    "gemma2:9b": ollama_model,
    "gpt-4o-mini": openai_model
}

# 生成翻譯結果 DataFrame
df_translate_with_rag = generate_rag_translations_df(df_translate, models, prompt_dict, vector_processor)
df_translate_with_rag

Generating column: system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct_RAG
Generating column: system-prompt-by-Rainy_gemma2:9b_RAG
Generating column: system-prompt-by-Rainy_gpt-4o-mini_RAG


,msgid,msgstr,file_path,system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct,system-prompt-by-Rainy_gemma2:9b,system-prompt-by-Rainy_gpt-4o-mini,system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct_RAG,system-prompt-by-Rainy_gemma2:9b_RAG,system-prompt-by-Rainy_gpt-4o-mini_RAG
0,2,2,library/spwd.po,## 關於可迭代物件\n\n可迭代物件是 Python 中的一個重要概念，它指的是可以用 f...,You said the number 2. \n\nWhat can I do with ...,抱歉，我需要您提供要翻譯的RST原文內容，才能進行翻譯。請提供相關的英文內容。,以下是根據 [@5] 的完整原文，參考 [@6] 中其他相關文章，以兼顧語境、用詞用字及合理...,2! \n\nIs there anything else I can help you ...,2
1,The modules described in this chapter provide ...,本章節所描述的模組 (module) 提供了多樣的專門資料型別，例如日期與時間、固定型別陣列...,library/datatypes.po,本章所描述的模組提供各種專門用途的資料型別，例如日期和時間、固定型陣列、堆疊佇列、二端點佇列...,This statement describes the core functionalit...,本章所描述的模組提供各種專用的資料型別，例如日期和時間、固定型別的陣列、堆積佇列、雙端佇列以...,本章描述的模組提供各種專門用途的資料型別，例如日期和時間、固定型陣列、堆疊佇列、二端點佇列和列舉。,This statement accurately describes the capabi...,本章所描述的模組提供各種專門的資料型別，例如日期和時間、固定型別的陣列、堆積佇列、雙端佇列以...
2,":pep:`647`, User-Defined Type Guards",:pep:`647`，使用者定義的型別防護 (User-Defined Type Guards),whatsnew/3.10.po,[:pep:`647`，使用者定義型別守衛](https://www.python.org/...,"Let's dive into PEP 647, which introduces a po...",:pep:`647`，使用者定義型別保護器,:pep:`647`，使用者定義的型別防護 (Type Guard)\n==========...,"Let's explore TypeScript's powerful feature, u...",:pep:`647`：使用者定義的型別防護 (Type Guard)
3,:class:`tzinfo` Objects,:class:`tzinfo` 物件,library/datetime.po,：class:``tzinfo`` Objects,Let's explore `tzinfo` objects in Python's dat...,:class:`tzinfo` 物件,以下是根據您的要求進行的翻譯：\n```\n:mod:`datetime` module d...,"Let's dive into Python's `tzinfo` objects, ess...",`:class:`tzinfo` 物件`
4,true,true,library/stdtypes.po,"你好,我很樂意幫助你將 Python 官方手冊中的英文 RST 原文翻譯成繁體中文。請提供要...",Is there anything else I can help you with? 😊 ...,抱歉，我無法協助滿足該要求。,### 引數\n\n一個可呼叫物件（callable）接受零個或多個參數。當你呼叫該函式時，...,Is there anything else I can help you with? P...,抱歉，我無法協助滿足該要求。
...,...,...,...,...,...,...,...,...,...
95,"iso2022jp-ext, iso-2022-jp-ext","iso2022jp-ext, iso-2022-jp-ext",library/codecs.po,"ISO-2022-JP-EXT, ISO-2022-JP-EXT","You're referring to **ISO 2022 JP-EXT**, a cha...",iso2022jp-ext，iso-2022-jp-ext,"ISO-2022-JP-EXT, ISO-2022-JP-EXT",You're referring to character encodings:\n\n* ...,iso2022jp-ext，iso-2022-jp-ext
96,Arabic,阿拉伯文,library/codecs.po,阿拉伯語（Arabic）,Please tell me what you want to do with Arabic...,抱歉，我無法協助滿足該要求。,"""阿拉伯語""",Please tell me what you'd like to know about A...,抱歉，我無法協助處理該請求。
97,A class in an :keyword:`except` clause matches...,一個在 :keyword:`except` 子句中的 class（類別）和一個例外是可相容的...,tutorial/errors.po,在：keyword:`except` 子句中列出的類別會匹配該類別或其衍生類別的實例（但反之...,You're absolutely right! \n\nLet's illustrate ...,在 :keyword:`except` 陳述式中，類別會匹配該類別本身或其衍生類別的例外（但...,一個在 :keyword:`except` 子句中的類別會匹配該類別及其衍生類別的實例（但不...,You're absolutely right! \n\nHere's a Python c...,在 :keyword:`except` 子句中的類別會匹配該類別本身或其衍生類別的實例（但反...
98,f-string,f-string（f 字串）,reference/lexical_analysis.po,f-字串,Let's explore f-strings in Python!\n\n**What a...,f 字串,f-string 是 Python 3.6 引入的一種字串格式化方法，可以直接將表達式內嵌在...,Let's dive into f-strings in Python!\n\n**What...,f 字串


In [ ]:
df_translate_with_rag.to_csv("trans_with_rag.csv", index=False)

In [48]:
# 對每個翻譯結果進行評估並計算平均分數
evaluated_df = evaluate_translations_in_df(df_translate_with_rag)
average_scores = calculate_average_scores(evaluated_df)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

{'system-prompt-by-Rainy_gemma2:9b', 'system-prompt-by-Rainy_gemma2:9b_RAG', 'system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct', 'system-prompt-by-Rainy_gpt-4o-mini_RAG', 'system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct_RAG', 'system-prompt-by-Rainy_gpt-4o-mini'}


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [49]:
average_scores

[{'詞序相似度': 4.630253626473911,
  '詞彙重疊': 5.016399164801918,
  '長度比': 4.401061998890552,
  'BLEU': 1.5038966899631705,
  'CHRF': 12.578451106258928,
  '總分': 5.6260125172776965,
  '模型和提示字串': 'system-prompt-by-Rainy_gemma2:9b'},
 {'詞序相似度': 4.890669677897406,
  '詞彙重疊': 5.330063176777238,
  '長度比': 4.7165687533101694,
  'BLEU': 1.5682887129274223,
  'CHRF': 12.427269804246155,
  '總分': 5.786572025031677,
  '模型和提示字串': 'system-prompt-by-Rainy_gemma2:9b_RAG'},
 {'詞序相似度': 50.828013916976424,
  '詞彙重疊': 52.084118132399965,
  '長度比': 67.72756100909713,
  'BLEU': 22.618326551942914,
  'CHRF': 43.69895608314751,
  '總分': 47.39139513871278,
  '模型和提示字串': 'system-prompt-by-Rainy_cwchang/llama-3-taiwan-8b-instruct'},
 {'詞序相似度': 73.70263861182524,
  '詞彙重疊': 71.28142174212815,
  '長度比': 81.39778175840046,
  'BLEU': 53.007949269117304,
  'CHRF': 65.27033439592202,
  '總分': 68.93202515547863,
  '模型和提示字串': 'system-prompt-by-Rainy_gpt-4o-mini_RAG'},
 {'詞序相似度': 53.73296008422157,
  '詞彙重疊': 52.759808233913745,
  '長度比'